In [1]:
import pandas as pd
import pyodbc
# File names and directories
survey_dir = 'J:/Projects/Surveys/HHTravel/Survey2019/Data/PSRC_2019_HTS_Deliverable_022020/Weighted_Data_022020/'
hh_file_name = '1_household.csv'
person_file_name = '2_person.csv'
vehicle_file_name = '3_vehicle.csv'
day_file_name = '4_day.csv'
trip_file_name = '5_trip.csv'

#sql_conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=AWS-PROD-SQL\COHO;DATABASE=HouseholdTravelSurvey2019;trusted_connection=true')
#trip_table_name = "HHSurvey.Trip"
#trip  = pd.read_sql('SELECT * FROM '+trip_table_name, con = sql_conn)

hh_weight_2017 = 'hh_wt_revised'
hh_weight_2019 = 'hh_wt_2019'
hh_weight_combined = 'hh_wt_combined'
hh_weight_names =[hh_weight_2017, hh_weight_2019, hh_weight_combined]

day_weight_2017 = 'hh_day_wt_revised'
day_weight_2019 = 'hh_day_wt_2019'
day_weight_combined = 'hh_day_wt_combined'
day_weight_names =[day_weight_2017, day_weight_2019, day_weight_combined]

trip_weight_2017 = 'trip_weight_revised'
trip_weight_2019 ='trip_wt_2019'
trip_weight_combined = 'trip_wt_combined'
trip_weight_names =[trip_weight_2017, trip_weight_2019, trip_weight_combined]

codebook_file = 'Combined_Codebook_022020.xlsx'
output_file_hh = 'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/hh_summary.xlsx'
output_file_person= 'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/person_summary.xlsx'
output_file_day= 'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/day_summary.xlsx'
output_file_trip= 'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/trip_summary.xlsx'

mode_simple_file =  'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/mode_simple.xlsx'
purpose_simple_file =  'C:/Users/SChildress/Documents/GitHub/travel-studies/2019/summary/purpose_simple.xlsx'

In [2]:
def merge_hh_person_trip(hh,person,trip):
    hh_person =pd.merge(hh, person, on= 'hhid', suffixes=['', 'person'], how ='right')
    hh_person_trip = pd.merge(hh_person, trip,on='personid', suffixes=['','trip'], how ='right')
    return hh_person_trip

def merge_hh_person(hh, person):
    hh_person =pd.merge(hh, person, on= 'hhid', suffixes=['', 'person'], how ='right')
    return hh_person


In [3]:
hh = pd.read_csv(survey_dir+hh_file_name, encoding = 'latin-1')
person= pd.read_csv(survey_dir+person_file_name, encoding = 'latin-1')
trip = pd.read_csv(survey_dir+trip_file_name, encoding = 'latin-1')
day= pd.read_csv(survey_dir+day_file_name, encoding='latin-1')

C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8,44,45,53,56,57,65,72,73,81) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,10,11,20,24,37,39,106,112,118,120,121,128) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (14,15,21,23,34,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
codebook_values = pd.read_excel(survey_dir+codebook_file, sheetname='Values')
codebook_variables =pd.read_excel(survey_dir+codebook_file, sheet_name='Data Overview')

C:\Users\SChildress\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [5]:
mode_simple= pd.read_excel(mode_simple_file)
purpose_simple = pd.read_excel(purpose_simple_file)

In [6]:
person_detail = merge_hh_person(hh, person)

In [7]:
person_trip = merge_hh_person_trip(hh,person,trip)

In [8]:
def lookup_codes(grouped_total, item):
    values=codebook_values.loc[codebook_values['variable']==item]
    variable =codebook_variables.loc[codebook_variables['variable']==item, 'description'].values[0]
    missing_values_df = pd.DataFrame({'variable': [item, item, item],'value':[-9999,-9998, 995], 'label':
                                         ['Missing:Technical Error', 'Missing: Non-response', 'Missing:Skip logic']})
    values = values.append(missing_values_df)
    summary=pd.merge(values, grouped_total,left_on='value', right_on=item)
    summary[item+'_variable']=variable
    return summary

In [9]:
person_commute= person_detail.groupby('commute_mode').sum()[hh_weight_2019].reset_index()

In [10]:
person_commute =lookup_codes(person_commute, 'commute_mode')

In [11]:
person_commute

,variable,value,label,commute_mode,hh_wt_2019,commute_mode_variable
0,commute_mode,1,Drive alone,1.0,1.368811e+06,Usual way of commuting to current location/office
1,commute_mode,2,Carpool ONLY with other household members,2.0,1.615599e+05,Usual way of commuting to current location/office
2,commute_mode,3,Carpool with other people not in household (ma...,3.0,2.387081e+04,Usual way of commuting to current location/office
3,commute_mode,4,Motorcycle/moped/scooter,4.0,0.000000e+00,Usual way of commuting to current location/office
4,commute_mode,5,Vanpool,5.0,2.375706e+04,Usual way of commuting to current location/office
5,commute_mode,6,Bicycle or e-bike,6.0,6.492544e+04,Usual way of commuting to current location/office
6,commute_mode,7,"Walk, jog, or wheelchair",7.0,3.286490e+04,Usual way of commuting to current location/office
7,commute_mode,8,Bus (public transit),8.0,1.646594e+05,Usual way of commuting to current location/office
8,commute_mode,9,Private bus or shuttle,9.0,1.151994e+04,Usual way of commuting to current location/office
9,commute_mode,10,Paratransit,10.0,0.000000e+00,Usual way of commuting to current location/office


In [12]:
person_commute_county = person_detail.groupby(['commute_mode', 'sample_county']).sum()[hh_weight_2019].reset_index()

In [13]:
person_modes_cnty =lookup_codes(person_commute_county, 'commute_mode')

In [14]:
person_modes_cnty

,variable,value,label,commute_mode,sample_county,hh_wt_2019,commute_mode_variable
0,commute_mode,1,Drive alone,1.0,King,6.897439e+05,Usual way of commuting to current location/office
1,commute_mode,1,Drive alone,1.0,Kitsap,9.274960e+04,Usual way of commuting to current location/office
2,commute_mode,1,Drive alone,1.0,Pierce,2.989401e+05,Usual way of commuting to current location/office
3,commute_mode,1,Drive alone,1.0,Snohomish,2.873773e+05,Usual way of commuting to current location/office
4,commute_mode,2,Carpool ONLY with other household members,2.0,King,9.007467e+04,Usual way of commuting to current location/office
5,commute_mode,2,Carpool ONLY with other household members,2.0,Kitsap,1.847255e+04,Usual way of commuting to current location/office
6,commute_mode,2,Carpool ONLY with other household members,2.0,Pierce,1.714205e+04,Usual way of commuting to current location/office
7,commute_mode,2,Carpool ONLY with other household members,2.0,Snohomish,3.587065e+04,Usual way of commuting to current location/office
8,commute_mode,3,Carpool with other people not in household (ma...,3.0,King,1.095037e+04,Usual way of commuting to current location/office
9,commute_mode,3,Carpool with other people not in household (ma...,3.0,Kitsap,2.241896e+03,Usual way of commuting to current location/office


In [16]:
person_modes_cnty.to_clipboard()

In [17]:
person_modes_puma = person_detail.groupby(['commute_mode', 'final_home_puma10']).sum()[hh_weight_2019].reset_index()
person_modes_puma =lookup_codes(person_modes_puma, 'commute_mode')
person_modes_puma.to_clipboard()